# ELEC 474 Prelab 
_By Matthieu Roux_ 

Student ID: 20013052

## Changing the image
You can change the image you will be playing with by changing the variable name in `img_path` below. By default we will be using the baboon image provided but you can play try the custom image by doing `img_path = my_image_path`

In [1]:
my_image_path = "my_image.png"
baboon_image_path = "baboon.png"

# edit this to change the image you want to play with
img_path = baboon_image_path

Imports that will be needed.
I am importing the `copy` package in order to do deep copies which is used to keep a good copy of the seed pixel.

In [2]:
import cv2
import numpy as np
import copy # used to create deep copies of the original pixel

In here the image is being imported and all the global variables are being initailized.

`segment_pixels` is the dictionary that is going to be used to find out which pixels are already in the segment. The use of a dcitionary ensures more performance when looking for existing pixels.

`next_pixel_stack` is the list of pixels that could be part of the region that are waiting to be processed.

In [3]:
img = cv2.imread(img_path)

x = y = red = green = blue = 0

segment_pixels = {}

next_pixel_stack=[]

threshold = 25

window_name = "image"

segment_color = [0, 255, 0] # should be a very bright green

### onMouse
The `onMouse` function handles the left click trigger on the image and obtains the coordinates of the click.

As instructed, the pixel that is clicked is pushed into the `next_pixel_stack` and will be used as the seed pixel.

In [4]:
def onMouse(event,_x,_y,flags,param):
    global x, y, green, blue, red, img, next_pixel_stack
    if event == cv2.EVENT_LBUTTONDOWN:
        green = img[_y,_x,0]
        blue = img[_y,_x,1]
        red = img[_y,_x,2]
        x = _x
        y = _y
        
        next_pixel_stack.append((_y,_x))
        segmentation()

## The Segmentation

The segmentation follows the following algorithm:

   1. Take the only pixel in the `next_pixel_stack` and set it as the `seed_pixel`
    
   2. Pop the next pixel in the stack
    
   3. If the pixel is already in the segment, skip to step 7
    
   4. Run the homogeneity function to see if the pixel belongs in the region, if the pixel is **not** homogeinous go to step 7
    
   5. Add the pixel in the region and update its color
    
   6. Find the pixel's neighbours using **4-connectivity** adn add them to the stack
    
   7. If there are pixels left in the stack go to step 2
  
### Notes

I was not sure if we were supposed to reset the regions after each click, but I implemented code to reset the image (by pressing the `r` key) below to reset all regions.

In [5]:
def segmentation():
    
    # get all global variables
    global x, y, green, blue, red, img, next_pixel_stack, segment_pixels, segment_color
    
    # set the seed pixel
    seed_pixel = copy.deepcopy(img[next_pixel_stack[0]])
    
    while len(next_pixel_stack) > 0:
        
        # pop the next pixel in the stack
        pixel = next_pixel_stack.pop()
        
        # if the pixel is already in the segment, skip it
        if pixel in segment_pixels:
            continue
        # run the homogeneity function to see if the pixel belongs in the region
        if is_homogeneous(seed_pixel, img[pixel]):
            
            # Add the pixel in the region dicitonary
            segment_pixels[pixel] = True
            
            # update the pixel color
            img[pixel] = segment_color
            
            # find the next potential pixels and add them to the stack
            next_pixels = get_next_pixels(pixel)
            next_pixel_stack.extend(next_pixels)

### Homogneity function
My homogeneity function ensures that the pixel belongs in the region by ensuring that all subpixel values of the analyzed pizel have a maximum difference of `threshold` compared to the `seed_pixel`.

Algorithm:
1. Compute the difference between the blue value of the pixel and the blue value of the seed pixel
2. Compute the difference between the green value of the pixel and the green value of the seed pixel
3. Compute the difference between the red value of the pixel and the red value of the seed pixel
(the order in those steps do not matter)
4. Ensure that the differences obtained in steps 1 to 3 are below or equal to the `threshold`

In [6]:
def get_diff(value1, value2):
    if value1 > value2:
        diff = value1 - value2
    else:
        diff = value2 - value1
    return diff
    
def is_homogeneous(seed_pixel, pixel):
    global img, threshold
    diffs = [get_diff(seed_pixel[i], pixel[i]) for i in range(3)]
    homogeneous = diffs[0] <= threshold and diffs[1] <= threshold and diffs[2] <= threshold
    return homogeneous

In [7]:
# get_next_pixels finds the next potential pixels in the region using 4-connectivity
def get_next_pixels(pixel):
    global img
    max_x = img.shape[1] - 1
    max_y = img.shape[0] - 1
    next_pixels = []
    if pixel[0] != 0:
        next_pixels.append((pixel[0] - 1, pixel[1]))
    if pixel[0] != max_y:
        next_pixels.append((pixel[0] + 1, pixel[1]))
    if pixel[1] != 0:
        next_pixels.append((pixel[0], pixel[1] - 1))
    if pixel[1] != max_x:
        next_pixels.append((pixel[0], pixel[1] + 1))
    return next_pixels

In [8]:
# on_trackbar is called when the trackbar is changed, it updates the threshold
def on_trackbar(val):
    global threshold
    threshold = val

## The Main Loop

The main loop runs until the `x` key is pressed, which will close the open cv window.

Keep in mind that the windiwo may open minimized and you may have to click on it in your taskbar.

Pressing `r` will reset the image.

In [9]:
cv2.namedWindow(window_name)

# Create the trackbar that will allow users to edit the threshold
max_threshold = 100
cv2.createTrackbar("Threshhold", window_name, threshold , max_threshold, on_trackbar)

while(True):
    # Detect mouse clicks
    cv2.setMouseCallback(window_name, onMouse)
    
    # Wait a little bit for the image to re-draw
    key = cv2.waitKey(1)
    cv2.imshow(window_name, img)
    
    # If an x is pressed, the window will close
    if key == ord("x"):
        break
        
    # If r is pressed, the image adn regions will reset.
    if key == ord("r"):
        img = cv2.imread(img_path)
        segment_pixels = {}

In [10]:
cv2.destroyAllWindows()